In [3]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib
import keras
from keras.datasets import cifar10,mnist
from keras.utils import np_utils
from keras.optimizers import SGD
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import os
import sys
sys.path.insert(0, '..')
from binarization_utils import *
from model_architectures import get_model

Using TensorFlow backend.
/Users/justinshao/opt/anaconda3/envs/research/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [4]:
dataset='MNIST'
Train=False
Evaluate=True
batch_size=100
epochs=200

In [5]:
def load_svhn(path_to_dataset):
	import scipy.io as sio
	train=sio.loadmat(path_to_dataset+'/train.mat')
	test=sio.loadmat(path_to_dataset+'/test.mat')
	extra=sio.loadmat(path_to_dataset+'/extra.mat')
	X_train=np.transpose(train['X'],[3,0,1,2])
	y_train=train['y']-1

	X_test=np.transpose(test['X'],[3,0,1,2])
	y_test=test['y']-1

	X_extra=np.transpose(extra['X'],[3,0,1,2])
	y_extra=extra['y']-1

	X_train=np.concatenate((X_train,X_extra),axis=0)
	y_train=np.concatenate((y_train,y_extra),axis=0)

	return (X_train,y_train),(X_test,y_test)

if dataset=="MNIST":
	(X_train, y_train), (X_test, y_test) = mnist.load_data()
	# convert class vectors to binary class matrices
	X_train = X_train.reshape(-1,784)
	X_test = X_test.reshape(-1,784)
	use_generator=False
elif dataset=="CIFAR-10":
	use_generator=True
	(X_train, y_train), (X_test, y_test) = cifar10.load_data()
elif dataset=="SVHN":
	use_generator=True
	(X_train, y_train), (X_test, y_test) = load_svhn('./svhn_data')

In [6]:
X_train=X_train.astype(np.float32)
X_test=X_test.astype(np.float32)
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
X_train /= 255
X_test /= 255
X_train=2*X_train-1
X_test=2*X_test-1


print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 784)
60000 train samples
10000 test samples


In [7]:
def get_binarized_model(dataset, resid_levels):
    model = get_model(dataset, resid_levels)
    
    for i in range(0,5):
        layer = model.get_layer(index = 3*i)
        gamma = np.mean([np.mean(a) for a in layer.get_weights()])
        model.get_layer(index = 3*i).set_weights([np.sign(a)*np.mean(np.abs(a)) for a in model.get_layer(index = 3*i).get_weights()])

    return model

In [19]:
resid_levels = 1
print('model performance:')
weights_path='models/'+dataset+'/'+str(resid_levels)+'_residuals.h5'
model=get_model(dataset,resid_levels)
model.load_weights(weights_path)
opt = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
#model.summary()
score=model.evaluate(X_test,Y_test,verbose=0)
print("with %d residuals, test loss was %0.4f, test accuracy was %0.4f"%(resid_levels,score[0],score[1]))
    
print('binarized model performance')
weights_path='models/'+dataset+'/'+str(resid_levels)+'_residuals.h5'
model=get_binarized_model(dataset,resid_levels)
model.load_weights(weights_path)
opt = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
#model.summary()
score=model.evaluate(X_test,Y_test,verbose=0)
print("with %d residuals, test loss was %0.4f, test accuracy was %0.4f"%(resid_levels,score[0],score[1]))

model performance:
with 1 residuals, test loss was 0.1192, test accuracy was 0.9790
binarized model performance


/Users/justinshao/opt/anaconda3/envs/research/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/justinshao/opt/anaconda3/envs/research/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


with 1 residuals, test loss was 0.1192, test accuracy was 0.9790


In [59]:
def bitFlip(a, p01, p10):
    '''if (isinstance(a, float)):
        p = tf.cond(a < 0, lambda: p01, lambda: p10)
        if np.random.rand() < p:
            return -1 * a
        else:
            return a
    else:
        return [bitFlip(b, p01, p10) for b in a]'''
    
    p = tf.cond(a < 0, lambda: p01, lambda: p10)
    if (np.random.rand() < p):
        return -1 * a
    else:
        return a

class Nonideal_sign(Layer):
    def __init__(self, levels=1,**kwargs):
        self.levels=levels
        super(Nonideal_sign, self).__init__(**kwargs)
    def build(self, input_shape):
        ars=np.arange(self.levels)+1.0
        ars=ars[::-1]
        means=ars/np.sum(ars)
        self.means=[K.variable(m) for m in means]
        self.trainable_weights=self.means
    def call(self, x, mask=None):
        resid = x
        out_bin=0
        out=binarize(resid)*K.abs(self.means[0])
        out_bin=out_bin+out
        resid=resid-out
        return tf.map_fn(lambda a: bitFlip(a, 0, 0), out)

    def get_output_shape_for(self,input_shape):
        return input_shape
    def compute_output_shape(self,input_shape):
        return input_shape
    def set_means(self,X):
        means=np.zeros((self.levels))
        means[0]=1
        resid=np.clip(X,-1,1)
        approx=0
        for l in range(self.levels):
            m=np.mean(np.absolute(resid))
            out=np.sign(resid)*m
            approx=approx+out
            resid=resid-out
            means[l]=m
            err=np.mean((approx-np.clip(X,-1,1))**2)

        means=means/np.sum(means)
        sess=K.get_session()
        sess.run(self.means.assign(means))

In [60]:
batch_norm_eps=1e-4
batch_norm_alpha=0.1#(this is same as momentum)

model=Sequential()
model.add(binary_dense(n_in=784,n_out=256,input_shape=[784]))
model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
model.add(Nonideal_sign(levels=resid_levels))
model.add(binary_dense(n_in=int(model.output.get_shape()[1]),n_out=256))
model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
model.add(Nonideal_sign(levels=resid_levels))
model.add(binary_dense(n_in=int(model.output.get_shape()[1]),n_out=256))
model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
model.add(Nonideal_sign(levels=resid_levels))
model.add(binary_dense(n_in=int(model.output.get_shape()[1]),n_out=256))
model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
model.add(Nonideal_sign(levels=resid_levels))
model.add(binary_dense(n_in=int(model.output.get_shape()[1]),n_out=10))
model.add(BatchNormalization(axis=-1, momentum=batch_norm_alpha, epsilon=batch_norm_eps))
model.add(Activation('softmax'))

model.load_weights(weights_path)
opt = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
#model.summary()
score=model.evaluate(X_test,Y_test,verbose=0)
print("with %d residuals, test loss was %0.4f, test accuracy was %0.4f"%(resid_levels,score[0],score[1]))


ValueError: Shape must be rank 0 but is rank 1 for 'nonideal_sign_44/map/while/cond/Switch' (op: 'Switch') with input shapes: [256], [256].

In [39]:
theee

NameError: name 'theee' is not defined